# How to Use the Web Services API for Dataset Searching
***

## Overview:   
The NASA Goddard Earth Sciences Data and Information Services Center (GES DISC) has developed an Application Program Interface (API) that is intended for users who prefer to employ our data search and subsetting services using scripts instead of the Web browser interface. The API protocol allows users to find the datasets and data granules they need, and to download any desired subsets. Information is passed back and forth in JavaScript Object Notation (JSON) format. 

## Example:  
This example code demonstrates how to use the API to search GES DISC data collections. The code prompts the user for a search string, submits the search to the GES DISC server, and then retrieves the search results. The response from the server contains a large collection of metadata for each dataset that matches the search string. We provide some examples below that will print out selected pieces of information for each item that might be relevant for a) narrowing down the search, b) finding documentation, or c) setting up another API script to perform a subsetting request. The final example shows how to write out the entire search response to a local file.

### Prerequisites:  
This example code is written in Python3 and requires these libraries: sys, json, datetime, urllib3, and certifi. 

### Procedure
The first step is to import the required Python libraries. If any of the following import commands fail, check the local Python environment and install any missing packages. 

In [ ]:
import sys 
import json
import datetime
import certifi
import urllib3

Initialize the urllib PoolManager and set the base URL for dataset search API requests

In [ ]:
# Create a PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())

# Set the URL for the GES DISC API endpoint for dataset searches
url = 'https://disc.gsfc.nasa.gov/service/datasets/jsonwsp'

The user will be prompted for the desired search parameters -- the syntax is similar to what would be employed when searching with the Web browser interface. The search request is posted to the GES DISC server, and the response is checked for any errors. If there are errors, the user gets another prompt. When the results from a successful search have been retrieved, the script prints "OK", followed by the number of items in the search results.

Run the following block of code and try these four search strings, noting how the number of results diminishes with each additional keyword:  
_merra-2  
merra-2 hourly  
merra-2 hourly 2d  
merra-2 hourly 2d instantaneous_

In [ ]:
# Prompt for search string keywords
# This will keep looping and prompting until search returns an error-free response
done = False
while done is False :
  myString=''
  while len(myString) < 1 : 
    myString = input("Enter search keywords: ")

  # Set up the JSON WSP request for API method: search
  search_request = {
    'methodname': 'search',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {'search': myString}
  }

  # Submit the search request to the GES DISC server
  hdrs = {'Content-Type': 'application/json',
          'Accept': 'application/json'}
  data = json.dumps(search_request)
  r = http.request('POST', url, body=data, headers=hdrs)
  response = json.loads(r.data)

  # Check for errors
  if response['type']=='jsonwsp/fault' :
    print('ERROR! Faulty request. Please try again.')
  else : 
    done = True
print('OK')

# Indicate the number of items in the search results
total = response['result']['totalResults']
if total == 0 :
    print('Zero items found')
elif total == 1 : 
    print('1 item found')
else :          
    print('%d items found' % total)

The response from the server contains <b><em>all</em></b> the metadata for each item in the list of dataset search results. The entire response contains much more information than users will likely need. The four examples below illustrate how to extract and print out a few critical pieces of metadata for each search result item, which could prove helpful for narrowing down the dataset search or developing API subsetting scripts. Please see <em>"How to Use the Web Services API for Subsetting MERRA-2 Data"</em> for detailed instructions on subsetting with the API. 

The code below prints out the DatasetID (which is needed for any API subset requests) and the Label, a more human-readable description of the dataset. 

In [ ]:
# Report on the results: DatasetID and Label
if total > 0 :
    for item in response['result']['items']:
        print('%-20s  %s' % (item['dataset']['id'], item['dataset']['label']))

The code below prints out the DatasetID along with the starting and ending dates. 

In [ ]:
# Report on the results: DatasetID, StartDate, and EndDate
if total > 0 :
    for item in response['result']['items']:
        start = datetime.datetime.utcfromtimestamp(int(item['startDate']/1000))
        end   = datetime.datetime.utcfromtimestamp(int(item['endDate']/1000))
        print('%-20s  Start Date = %s    End Date = %s' % (item['dataset']['id'], start, end))                        

The code below prints out the DatasetID and the Landing Page URL, which is the centralized access point for all the web-based information and services associated with that dataset.

In [ ]:
# Report on the results: DatasetID and Landing Page URL
if total > 0 :
    for item in response['result']['items']:
        print('%-20s  %s' % (item['dataset']['id'], item['link'])) 

The code below prints out the DatasetID and a list of variable names and descriptions. The DatasetID and the variable name are pieces of metadata that are used for making subset requests with the API. 

In [ ]:
# Report on the results: DatasetID and variable subsetting information
varSubset = False
for item in response['result']['items']:
    # Check for subset services
    if item['services']['subset']: 
        for ss in item['services']['subset']:
            # make sure variable subsetting is supported
            if 'variables' in ss['capabilities'] and 'dataFields' in ss :
                print('The %s service supports variable subsetting for %s' % 
                      (ss['agentConfig']['agentId'],item['dataset']['id']))
                print('Variable names are:')
                varSubset = True
                # Print a list of variable names and descriptions
                for var in ss['dataFields']:
                    print(var['value'])
                print()
if varSubset is False: 
    print('Variable subsetting is not available for %s' % item['dataset']['id'])


The complete response can be written to a local text file in case the entire metadata record is needed. 

In [ ]:
# Write out the complete results to a file
fname='my_search_results.txt'
f = open(fname,'w')
f.write(json.dumps(response, indent=2, sort_keys=True))
f.close()
print('Complete metadata for each search result item has been written out to '+fname)

**Additional Info:**  
[Complete reference documentation on the GES DISC Dataset Service API](https://disc.gsfc.nasa.gov/service/datasets)  
[How to Use the Web Services API for Subsetting](https://disc.gsfc.nasa.gov/information/howto?keywords=API&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Subsetting)  
[How to Use the Web Services API for Subsetting MERRA-2 Data](https://disc.gsfc.nasa.gov/information/howto?keywords=api&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Subsetting%20MERRA-2%20Data)  
[The GES DISC guide to data access](https://disc.gsfc.nasa.gov/data-access)

<font size="1">THE SUBJECT FILE IS PROVIDED "AS IS" WITHOUT ANY WARRANTY OF ANY KIND, EITHER EXPRESSED, IMPLIED, OR STATUTORY, INCLUDING, BUT NOT LIMITED TO, ANY WARRANTY THAT THE SUBJECT FILE WILL CONFORM TO SPECIFICATIONS, ANY IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, OR FREEDOM FROM INFRINGEMENT, ANY WARRANTY THAT THE SUBJECT FILE WILL BE ERROR FREE, OR ANY WARRANTY THAT DOCUMENTATION, IF PROVIDED, WILL CONFORM TO THE SUBJECT FILE. THIS AGREEMENT DOES NOT, IN ANY MANNER, CONSTITUTE AN ENDORSEMENT BY GOVERNMENT AGENCY OR ANY PRIOR RECIPIENT OF ANY RESULTS, RESULTING DESIGNS, HARDWARE, SOFTWARE PRODUCTS OR ANY OTHER APPLICATIONS RESULTING FROM USE OF THE SUBJECT FILE. FURTHER, GOVERNMENT AGENCY DISCLAIMS ALL WARRANTIES AND LIABILITIES REGARDING THIRD-PARTY SOFTWARE, IF PRESENT IN THE SUBJECT FILE, AND DISTRIBUTES IT "AS IS."</font>